<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/2021notebooks/2021_1104animals_word_association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
* date: 2021_1103
* author: 浅川伸一
* license: MIT
* filename: 2021_1104animals_word_association.ipynb
---

# 動物の名前から部分空間を作成した場合の射影行列による動物の近接ベクトル

In [ ]:
# -*- coding: utf-8 -*-
# 自作ライブラリのインストール，今回は，日本語 wikipedia を使って訓練した自作の word2vec を読み込みためだけに使用
!git clone https://github.com/project-ccap/ccap.git

import ccap 
w2v = ccap.ccap_w2v().w2v  # word2vec の読み込み，実際には内部で gensim の `gensim.models.Word2Vec` に読み込ませている

In [ ]:
import numpy as np
# ここから先がモデル
# 動物を上の情報から動物の名前らしき項目だけ，勝手に選ぶ
#animals = ['イヌ', 'インコ', 'ウォッチング', 'ウサギ', 'カワウソ', 
#           'キジ', 'クジラ', 'サメ', 'スズメ', 'ティラノサウルス', 
#           'トカゲ', 'ネズミ', 'バクテリア', 'ヒト', 'フクロウ', 
#           'プランクトン', 'ヘビ', 'ムツゴロウ', 'ヤンバルクイナ', 'ワニ']
animals = ['おたまじゃくし', 'アオウミガメ', 'アオサギ', 'アオダイショウ', 'アカアシシギ', 
           'アカエイ', 'アカオネッタイチョウ', 'アカガエル', 'アカギツネ', 'アカゲザル', 
           'アカゲラ', 'アカシカ', 'アカテガニ', 'アカネズミ', 'アカボウクジラ', 'アカントステガ', 
           'アクロカントサウルス', 'アザラシ', 'アシカ', 'アジサシ', 'アナウサギ', 
           'アナグマ', 'アノマロカリス', 'アフリカゾウ', 'アフリカツメガエル', 'アブラツノザメ', 
           'アホウドリ', 'アマガエル', 'アマツバメ', 'アマモ', 'アミメニシキヘビ', 'アメフラシ', 
           'アメリカバイソン', 'アメリカワシミミズク', 'アメンボ', 'アメーバ', 'アユ', 
           'アライグマ', 'アリクイ', 'アリゲーターガー', 'アリジゴク', 'アルマジロ', 'アロサウルス', 
           'アロワナ', 'アンキロサウルス', 'アンコウ', 'アンテオサウルス', 'アンドリューサルクス', 
           'アンモナイト', 'アーケオプテリクス', 'アーケロン', 'アースロプレウラ', 'イエネコ', 
           'イクチオサウルス', 'イクチオステガ', 'イグアナ', 'イグアノドン', 'イサキ', 'イシガイ', 
           'イシサンゴ', 'イソギンチャク', 'イタセンパラ', 'イタチザメ', 'イッカク', 'イヌ', 
           'イヌワシ', 'イノシシ', 'イボニシ', 'イモムシ', 'イモリ', 'イリエワニ', 
           'イリオモテヤマネコ', 'イルカ', 'イロワケイルカ', 'イワシ', 'インコ', 'インドガビアル', 
           'インドクジャク', 'インドハッカ', 'ウイルス', 'ウグイ', 'ウサギ', 'ウシ', 
           'ウシガエル', 'ウズラ', 'ウツボ', 'ウナギ', 'ウバザメ', 'ウマ', 'ウミウシ', 
           'ウミガメ', 'ウミクワガタ', 'ウミサソリ', 'ウミタナゴ', 'ウミツバメ', 'ウミニナ', 
           'ウミヘビ', 'ウミユリ', 'ウンピョウ', 'エイリアン', 'エウステノプテロン', 'エゾヒグマ', 
           'エダフォサウルス', 'エチゼンクラゲ', 'エナガ', 'エピオルニス', 'エボシガイ', 'エミュー', 
           'オウムガイ', 'オオウナギ', 'オオウミガラス', 'オオカミ', 'オオコウモリ', 'オオサンショウウオ', 
           'オオセ', 'オオタカ', 'オオツノシカ', 'オオトカゲ', 'オオハシ', 'オオハシウミガラス', 'オオバン', 
           'オオフラミンゴ', 'オオマリコケムシ', 'オオメジロザメ', 'オオヤマネコ', 'オキアミ', 'オジロジカ', 
           'オジロワシ', 'オタマジャクシ', 'オットセイ', 'オナガガモ', 'オニアンコウ', 'オニオオハシ', 
           'オニヒトデ', 'オポッサム', 'オマキザル', 'オランウータン', 'オルガネラ', 'オヴィラプトロサウルス', 
           'カイウサギ', 'カイギュウ', 'カイメン', 'カエル', 'カギムシ', 'カサゴ', 'カジキ', 'カタクチイワシ', 
           'カタツムリ', 'カダヤシ', 'カッコウ', 'カツオクジラ', 'カツオドリ', 'カツオノエボシ', 'カナダガン', 
           'カニ', 'カニクイアザラシ', 'カニクイザル', 'カニダマシ', 'カバ', 'カピバラ', 'カブトガニ', 
           'カブトムシ', 'カマキリ', 'カメ', 'カメムシ', 'カモ', 'カモノハシ', 'カヤネズミ', 'カライワシ', 
           'カラシン', 'カラス', 'カルカロドントサウルス', 'カルガモ', 'カレイ', 'カワウソ', 'カワガラス', 
           'カワゲラ', 'カワスズメ', 'カワセミ', 'カワニナ', 'カワハギ', 'カワラヒワ', 'カンガルー', 
           'ガチョウ', 'キジ', 'キジバト', 'キセルガイ', 'キタキツネ', 'キタリス', 'キツツキ', 'キツネ', 
           'キツネザル', 'キバタン', 'キュウリウオ', 'キンクロハジロ', 'キンチャクダイ', 'ギガノトサウルス', 
           'ギフチョウ', 'ギンダラ', 'ギンブナ', 'クサガメ', 'クサフグ', 'クジラ', 'クニマス', 'クマ', 
           'クマネズミ', 'クマノミ', 'クモ', 'クモザル', 'クラゲ', 'クラーケン', 'クリオネ', 'クリダグナサス', 
           'クリーパー', 'クルマエビ', 'クロツヤムシ', 'クロトキ', 'クロナガアリ', 'クロマグロ', 'クワガタムシ', 
           'グッピー', 'グリプトドン', 'グリーンイグアナ', 'グンカンドリ', 'ケツァルコアトルス', 'ケンミジンコ', 
           'ケンムン', 'ケープハイラックス', 'ケープペンギン', 'ゲッチョ', 'ゲラダヒヒ', 'ゲンゴロウ', 
           'コアジサシ', 'コイ', 'コウイカ', 'コウノトリ', 'コウモリ', 'コオロギ', 'コクチョウ', 
           'コゲラ', 'コサギ', 'コジュケイ', 'コノドント', 'コビトカバ', 'コブハクチョウ', 'コモドオオトカゲ', 
           'コンゴウインコ', 'コンプソグナトゥス', 'ゴイサギ', 'ゴカイ', 'ゴキブリ', 'ゴマフアザラシ', 
           'ゴルゴサウルス', 'ゴルゴノプス', 'ゴンズイ', 'ゴンドウクジラ', 'サイチョウ', 'サカマキガイ', 
           'サケ', 'サケイ', 'サソリ', 'サメ', 'サル', 'サワガニ', 'サンゴ', 'サンショウウオ', 'ザトウクジラ', 
           'ザリガニ', 'シアノバクテリア', 'シカ', 'シカクナマコ', 'シギダチョウ', 'シクリッド', 'シジュウカラ', 
           'シノカリオプテリクス', 'シノサウロプテリクス', 'シファクティヌス', 'シマウマ', 'シマフクロウ', 
           'シャチ', 'シュモクザメ', 'ショウジョウバエ', 'シラウオ', 'シラミ', 'シロアリ', 'シロイルカ', 
           'シロウオ', 'シロサイ', 'シロナガスクジラ', 'シーサーペント', 'シーラカンス', 'ジャガーネコ', 
           'ジュゴン', 'ジリス', 'ジンベエザメ', 'スイギュウ', 'スカフォグナトゥス', 'スクミリンゴガイ', 
           'スジイルカ', 'スジエビ', 'スズガモ', 'スズメ', 'スズメガ', 'スズメダイ', 'スティラコサウルス', 
           'ステゴサウルス', 'ストロマトライト', 'スナギンチャク', 'スナネズミ', 'スナメリ', 'スネール', 
           'スピノサウルス', 'スミロドン', 'スローロリス', 'セイウチ', 'セイヨウミツバチ', 'セグロカモメ', 
           'セミクジラ', 'センザンコウ', 'ソコダラ', 'ゾウ', 'ゾウアザラシ', 'ゾウリムシ', 'タイコウチ', 
           'タガメ', 'タキタロウ', 'タコ', 'タスマニアデビル', 'タツノオトシゴ', 'タナゴ', 'タニシ', 
           'タヌキ', 'タンチョウ', 'ダイオウイカ', 'ダイサギ', 'ダチョウ', 'ダツ', 'ダマジカ', 'ダルマザメ', 
           'ダンクルオステウス', 'ダンゴムシ', 'チューブワーム', 'チョウ', 'チョウザメ', 'チョウチョウウオ', 
           'チョウチンアンコウ', 'チリモン', 'チンチラ', 'チンパンジー', 'ツェツェバエ', 'ツキノワグマ', 
           'ツチガエル', 'ツチスドリ', 'ツチノコ', 'ツチブタ', 'ツノザメ', 'ティラノサウルス', 'ティロサウルス', 
           'テッポウエビ', 'テナガエビ', 'テナガザル', 'テノントサウルス', 'テミンク', 'テンジクネズミ', 
           'テントウムシ', 'ディアデクテス', 'ディキノドン', 'ディノケファルス', 'ディモルフォドン', 
           'デイノテリウム', 'デイノニクス', 'デジモン', 'デスモスチルス', 'デルフチア・アシドボランス', 
           'トカゲ', 'トガリネズミ', 'トゲウオ', 'トビケラ', 'トビハゼ', 'トリガイ', 'トリケラトプス', 
           'トルヴォサウルス', 'トロオドン', 'ドクトカゲ', 'ドジョウ', 'ドブネズミ', 'ドロマエオサウルス', 
           'ドードー', 'ナガスクジラ', 'ナキウサギ', 'ナマケモノ', 'ナマコ', 'ナマズ', 'ナメクジ', 
           'ニゴイ', 'ニザダイ', 'ニシツノメドリ', 'ニシローランドゴリラ', 'ニジマス', 'ニタリクジラ', 
           'ニホンアカガエル', 'ニホンイシガメ', 'ニホンウナギ', 'ニホンカナヘビ', 'ニホンカワウソ', 
           'ニホンザリガニ', 'ニホンザル', 'ニホントカゲ', 'ニホンヤモリ', 'ニューギニアヤリガタリクウズムシ', 
           'ニワトリ', 'ヌタウナギ', 'ヌマエビ', 'ネコ', 'ネズッポ', 'ネズミ', 'ネズミイルカ', 'ネズミザメ', 
           'ネッシー', 'ノウサギ', 'ノコギリエイ', 'ノネズミ', 'ノーザンパイク', 'ハイイロガン', 'ハイギョ', 
           'ハエ', 'ハオリムシ', 'ハキリアリ', 'ハクジラ', 'ハクビシン', 'ハゲワシ', 'ハコフグ', 'ハシビロガモ', 
           'ハシビロコウ', 'ハジラミ', 'ハゼ', 'ハタネズミ', 'ハダカイワシ', 'ハダカデバネズミ', 'ハチ', 
           'ハチドリ', 'ハツカネズミ', 'ハト目', 'ハナムグリ', 'ハリネズミ', 'ハリモグラ', 'ハーテビースト', 
           'バイカルアザラシ', 'バクテリア', 'バッタ', 'バンドウイルカ', 'パキケトゥス', 'パキケファロサウルス', 
           'パレイアサウルス', 'ヒキガエル', 'ヒクイドリ', 'ヒグマ', 'ヒゲクジラ', 'ヒツジ', 'ヒト', 'ヒトデ', 
           'ヒドリガモ', 'ヒナモロコ', 'ヒヒ', 'ヒプシロフォドン', 'ヒヨケザル', 'ヒヨドリ', 'ヒラメ',
           'ピコプランクトン', 'ピラニア', 'ピラルクー', 'ピーシーズ', 'フェイゾン', 'フエガラス', 
           'フォルスラコス', 'フクロウ', 'フクロウオウム', 'フクロオオカミ', 'フクロネコ', 'フクロネズミ', 
           'フクロモモンガ', 'フグ', 'フジツボ', 'フデイシ', 'フルマカモメ', 'ブタ', 'ブチハイエナ', 
           'ブッポウソウ', 'ブラキオサウルス', 'プテラノドン', 'プテロダクティルス', 'プラナリア', 
           'プランクトン', 'プレシオサウルス', 'プレソドン', 'プレーリードッグ', 'プロガノケリス', 
           'プロトプラスト', 'ヘコアユ', 'ヘスペロルニス', 'ヘビ', 'ヘビクイワシ', 'ベニイロフラミンゴ', 
           'ベニザケ', 'ベンガルトラ', 'ベントス', 'ペッカリー', 'ペット', 'ペラゴルニス', 'ペリカン', 
           'ペリット', 'ペンギン', 'ホウボウ', 'ホオダレムクドリ', 'ホシハジロ', 'ホシムクドリ', 'ホタル', 
           'ホタルイカ', 'ホッキョクギツネ', 'ホッキョクグマ', 'ホホジロザメ', 'ホラアナグマ', 'ホライモリ', 
           'ホワイトバス', 'ホンソメワケベラ', 'ボノボ', 'ボラジョ', 'ポリプテルス', 'マイワシ', 'マガモ', 
           'マジュンガサウルス', 'マダイ', 'マダコ', 'マダニ', 'マッコウクジラ', 'マナヅル', 'マナティー', 
           'マハゼ', 'マプサウルス', 'マルハナバチ', 'マレーグマ', 'マンジュウダイ', 'マンモス', 
           'マーモセット', 'マーモット', 'マーレーコッド', 'ミクロラプトル', 'ミサゴ', 'ミシシッピアカミミガメ', 
           'ミジンコ', 'ミズガメ', 'ミズナギドリ', 'ミツバチ', 'ミトコンドリア', 'ミドリムシ', 
           'ミナミゾウアザラシ', 'ミミズ', 'ミミズク', 'ミュールジカ', 'ミーアキャット', 'ムカシトカゲ', 
           'ムカシトンボ', 'ムカデ', 'ムクドリ', 'ムササビ', 'ムツゴロウ', 'ムラサキウニ', 'メガテリウム', 
           'メガロドン', 'メジロザメ', 'メダカ', 'メトリオリンクス', 'メトロイド', 'メバル', 'モエビ', 
           'モクズガニ', 'モグラ', 'モササウルス', 'モツゴ', 'モノアラガイ', 'モモイロペリカン', 
           'モルモット', 'モンガラカワハギ', 'モーグリ', 'ヤギ', 'ヤシガニ', 'ヤツメウナギ', 'ヤドカリ', 
           'ヤノマミ', 'ヤブカ', 'ヤマアカガエル', 'ヤマアラシ', 'ヤマカガシ', 'ヤマシギ', 'ヤマトシジミ', 
           'ヤモリ', 'ヤンバルクイナ', 'ユスリカ', 'ユリカモメ', 'ヨコエソ', 'ヨコエビ', 'ヨコバイ', 
           'ヨシノボリ', 'ヨロイザメ', 'ヨーロッパヒメウ', 'ライチョウ', 'ラッコ', 'ランフォリンクス', 
           'リオプレウロドン', 'リャマ', 'ルリオーストラリアムシクイ', 'ルリビタキ', 'レイクトラウト', 
           'レイヨウ', 'レッサーパンダ', 'レプトケファルス', 'ロボク', 'ワシミミズク', 'ワタリガニ', 
           'ワニ', 'ワニトカゲギス', 'ワモンアザラシ', 'ワライカワセミ', 'ワラジムシ', 'ヴェロキラプトル', 
           '回虫', '始祖鳥', '山羊', '巻き貝', '巻貝', '栗鼠', '毛虫', '犬', '猫', '猿', '硬骨魚', 
           '蛇', '蛙', '蛾', '蜘蛛', '蠕虫', '豚', '象', '貝', '野ネズミ', '金魚', '鯨']

# 上で，勝手に選んだ animals の リストからどうぶつの射影行列 P を作成
animals = sorted(list(set(animals)))  # 重複を取り去るため 一旦 set にかけてから再度 list 化
n_animals = len(animals)
X = []          
for animal in animals:
    X.append(w2v[animal])
X = np.array(X)    

# 直上で作成した行列 X の射影行列を作成
XXT = X @ X.T                 # shpae(n_animals, n_animals) $XX^{T}$  行列の 2 乗
iXXT = np.linalg.inv(XXT)     # shape(n_animals, n_animals) $(XX^{T})^{-1}$ 行列の 2 乗の逆行列
XTiXTX = X.T @ iXXT           # shape(w2v_dim, n_animals)   $X^{T} (XX^{T})^{-1}$ 左から 
P = XTiXTX @ X                # shape(w2v_dim, w2v_dim)     $P=X^{T} (XX^{T})^{-1} X$

# 直交射影行列の作成
I = np.eye(len(P))            # shape(w2v_dim, w2v_dim)
C = I - P                     # shape(w2v_dim, w2v_dim)

CovP = X.T @ X                # shape(w2v_dim, w2v_dim)
CorP = np.corrcoef(X.T)

In [ ]:
# 試しに イヌ ベクトルを 射影行列に作用させ どうぶつの森空間ベクトル v を作成
word = 'イヌ'
x = np.array(w2v[word])

top_n = 5
print('直交射影空間上のベクトル: ', end='')
print([x[0] for x in w2v.similar_by_vector(np.dot(P,x), topn=top_n)])

print('直交補空間上のベクトル: ', end='')
print([x[0] for x in w2v.similar_by_vector(np.dot(C,x), topn=top_n)])

print('共分散行列を射影行列と見立てた場合の射影結果: ', end='')
print([x[0] for x in w2v.similar_by_vector(CovP @ x, topn=top_n)])

print('相関係数行列を射影行列と見立てた場合の射影結果: ', end='')
print([x[0] for x in w2v.similar_by_vector(CorP @ x, topn=top_n)])